# Financial Data Dashboard Plotly Dash

## Project Description: 

The below code creates a dashboard displaying 20-year financial statement data for 3,129 American publicly traded companies.

In [ ]:
# Import packages
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_auth

import numpy as np
import numpy_financial as npf
import pandas as pd
import re
from bs4 import BeautifulSoup
import requests

import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools

# Login details
USERNAME_PASSWORD_PAIRS = [['data','analyst']]

# Establish dash app
app = dash.Dash(external_stylesheets = [dbc.themes.BOOTSTRAP])

# Establish authorization login details
auth = dash_auth.BasicAuth(app,USERNAME_PASSWORD_PAIRS)

# Establish serever
server = app.server

# Create starting list of company names
doc1 = pd.read_csv('Data/Balance_Sheet_Data.csv')
doc1 = doc1[['company_name']]
doc1.columns = ['1']
doc1 = doc1['1'].unique()

new_doc1 = []
for i in doc1:
    string = str(i)
    new_doc1.append(string)
doc1 = new_doc1
doc1.sort()
search_options1 = [{'label': i, 'value': i} for i in doc1]

# Create starting list of financial statement labels applicable to current set value
doc2 = pd.read_csv('Data/Balance_Sheet_Data.csv')
doc2 = doc2.loc[(doc2['company_ticker'] == 'AAPL')]
doc2 = doc2[['field']]
doc2.columns = ['2']
doc2 = doc2['2'].unique()

new_doc2 = []
for i in doc2:
    string = str(i)
    new_doc2.append(string)
doc2 = new_doc2
doc2.sort()
search_options2 = [{'label': i, 'value': i} for i in doc2]
value = doc2[0]

# App layout
app.layout = html.Div([

    # Dynamic header by company
    html.Div([html.H2(id = 'header',
                      style = {'padding':10,
                               'margin':0,
                               'font-family':'Arial, Helvetica, sans-serif',
                               'background':'#1E90FF',
                               'color':'#FFFFFF',
                               'textAlign':'center'})]),

    # Radio buttons companies in or out of Value Index
    html.Div([dcc.RadioItems(id = 'rate',
                             options = [{'label': 'Balance Sheet', 'value': 1},
                                        {'label': 'Income Statement', 'value': 2},
                                        {'label': 'Cash-Flow Statement', 'value': 3}],
                             value = 1,
                             labelStyle = {'display': 'inline-block'},
                             inputStyle = {"margin-left": "10px"},
                             style = {'textAlign':'center',
                                      'padding-top':30,
                                      'padding-right': 15,
                                      'font-family':'Arial, Helvetica, sans-serif'})]),

    # Compnay dropdown menu
    html.Div([dbc.Row([dbc.Col(html.H4('Choose a Company')),
                       dbc.Col(html.H4(id = 'field_statement'))]),
              dbc.Row([dbc.Col(dcc.Dropdown(id = 'stock',
                               options = search_options1,
                               value = 'Apple Inc',
                               style = {'border-color': 'black'})),
                       dbc.Col(dcc.Dropdown(id = 'field',
                               options = search_options2,
                               value = value,
                               style = {'border-color': 'black'}))])],
             style = {'font-family':'Arial, Helvetica, sans-serif',
                      'padding-top':10,
                      'padding-right':'5%',
                      'padding-left':'5%',
                      'textAlign':'center'}),

    # Live market price for company stock
    html.Div([html.H5('Market Price:',
                      style = {'display':'inline-block',
                               'padding-left':20,
                               'padding-top':10}),

              html.H5(id = 'price',
                      style = {'display':'inline-block',
                               'padding-left':10}),

              # Live market price to earnings of company stock
              html.H5('Price To Earnings:',
                      style = {'display':'inline-block',
                               'padding-left':40,
                               'padding-top':10}),

              html.H5(id = 'PE',
                      style = {'display':'inline-block',
                               'padding-left':10}),

              # Current earnings per share of company stock in $
              html.H5('TTM Earnings Per Share:',
                      style = {'display':'inline-block',
                               'padding-left':40,
                               'padding-top':10}),

              html.H5(id = 'EPS',
                      style = {'display':'inline-block',
                               'padding-left':10}),

               # Current earnings per share of company stock %
              html.H5('TTM Earnings Per Share:',
                      style = {'display':'inline-block',
                               'padding-left':40,
                               'padding-top':10}),

              html.H5(id = 'EPS%',
                      style = {'display':'inline-block',
                               'padding-left':10})],
             style = {'font-family':'Arial, Helvetica, sans-serif',
                      'padding-top':30,
                      'textAlign':'center'}),

    # Divider
    html.Hr(style = {'padding-bottom' : 20,
                     'borderColor': 'black',
                     'width' : '90%'}),

    # Historical data for chosen field
    html.Div([dbc.Row([dbc.Col(dbc.Card(dcc.Graph(id = 'scat1'), body = True, color = "dark", outline = True),
                              width = {"size": 8,
                                       "order": 1}),
                       dbc.Col(dbc.Card(html.Div([html.H5('Compound Rate of Return Calculator',
                                                         style = {'padding-bottom': 10,
                                                                  'padding-top': 10}),
                                                  dbc.Row([dbc.Input(id = "input1", 
                                                                     placeholder = "Present Value, e.g. 1000", 
                                                                     type = "number")], 
                                                         style = {'padding-bottom': 5,
                                                                  'padding-left': 10,
                                                                  'padding-right': 10}),
                                                  dbc.Row([dbc.Input(id = "input2", 
                                                                     placeholder = "Future Value, e.g. 2000", 
                                                                     type = "number")],
                                                         style = {'padding-bottom': 5,
                                                                  'padding-left': 10,
                                                                  'padding-right': 10}),
                                                  dbc.Row([dbc.Input(id = "input3", 
                                                                     placeholder = "Years, e.g. 20", 
                                                                     type = "number")],
                                                         style = {'padding-bottom': 5,
                                                                  'padding-left': 10,
                                                                  'padding-right': 10}),
                                                  dbc.Row([html.Button('Submit', 
                                                                        id = 'submit-val')],
                                                         style = {'padding-top': 10,
                                                                  'padding-left': 10,
                                                                  'padding-right': 10}),
                                                  dbc.Row([html.Label('Compounded Rate of Return:'),
                                                           html.Div(id = 'output',
                                                                   style = {'padding-left': 10})],
                                                         style = {'padding-top': 10,
                                                                  'padding-left': 10,
                                                                  'padding-right': 10})]),
                                        body = True, 
                                        color = "dark", 
                                        outline = True),
                               width = {"size": 4,
                                        "order": 2})])],
            style = {'padding-left': 20,
                     'padding-right': 20}),

    # Historical price data chosen company
    html.Div([dbc.Row([dbc.Col(dbc.Card(dcc.Graph(id = 'scat2'), 
                                        body = True, 
                                        color = "dark", 
                                        outline = True),
                                width = {"size": 4,
                                         "order": 1}),

                       dbc.Col(dbc.Card(dcc.Graph(id = 'scat3'), 
                                        body = True, 
                                        color = "dark", 
                                        outline = True),
                               width = {"size": 4,
                                        "order": 2}),

                       dbc.Col(dbc.Card(dcc.Graph(id = 'scat4'), 
                                        body = True, 
                                        color = "dark", 
                                        outline = True),
                               width = {"size": 4,
                                        "order": 3})])],
            style = {'padding-left': 20,
                     'padding-right': 20,
                     'padding-top': 20,
                     'padding-bottom': 30}),

     # Company summary
     html.Div([html.H2('Company Summary',
                       style = {'padding':10,
                                'padding-top':10,
                                'margin':0,
                                'font-family':'Arial, Helvetica, sans-serif',
                                'background':'#1E90FF',
                                'color':'#FFFFFF',
                                'textAlign':'center'}),
               html.Div(id = 'summary',style={'padding':30,
                                              'font-family':'Arial, Helvetica, sans-serif',
                                              'line-height':30,
                                              'textAlign':'center',
                                              'fontSize':20})]),
    # Instructions
    html.Div([html.H2('Instructions',
                      style = {'padding':10,
                               'padding-top':10,
                               'margin':0,
                               'font-family':'Arial, Helvetica, sans-serif',
                               'background':'#1E90FF',
                               'color':'#FFFFFF',
                               'textAlign':'center'}),
              html.Div(html.P(["The dashboard provides data for 3,129 American publicly traded companies. \
              The purpose of the dashboard is to help the user observe 20-years of company financial data\
              regarding the balance sheet, income, and cash-flow statements. The user has the choice to choose\
              fields from each statement type by selecting the radio buttons above. Once the statement type has been\
              selected, the user can then choose a specific field, in that statement type, they want to view. \
              The 20-year historical data, for that field, will be displayed as a line chart (blue chart).",
              html.Br(),html.Br(),"For every company, the average price per share for the year ending in \
              in December, average annual diluted shares outstanding, and average market capitalization \
              (average price per share * average annual diluted shares outstanding) are also displayed as \
              line charts (orange charts).", html.Br(),html.Br(),"Additionally, the dashboard \
              provides live price, price to earnings, earnings per share, and company summary data."]),
                       style = {'padding':30,
                                'font-family':'Arial, Helvetica, sans-serif',
                                'line-height':30,
                                'textAlign':'center',
                                'fontSize':20})]),

    # Ending header
    html.Div([html.H2('',
                      style = {'padding':30,
                               'margin':0,
                               'font-family':'Arial, Helvetica, sans-serif',
                               'background':'#1E90FF',
                               'color':'#FFFFFF',
                               'textAlign':'center'})])],

    style = {'margin':0})

# App function for dropdown menu
@app.callback([Output('field','options'),
               Output('field','value')],
              [Input('rate','value'),
               Input('stock','value')])

def field_dropdown(rate, stock):

    # Radio buttons for financial statement type
    if rate == 1:
        # BalanceSheet
        doc1 = pd.read_csv('Data/Balance_Sheet_Data.csv')
        doc1 = doc1.loc[(doc1['company_name'] == stock)]
        doc1 = doc1[['field']]
        doc1 = doc1['field'].unique()

        new_doc1 = []
        for i in doc1:
            string = str(i)
            new_doc1.append(string)
        doc1 = new_doc1
        doc1.sort()
        search_options2 = [{'label': i, 'value': i} for i in doc1]
        value = doc1[0]
        
    elif rate == 2:
        # IncomeStatement
        doc1 = pd.read_csv('Data/Income_Statement_Data.csv')
        doc1 = doc1.loc[(doc1['company_name'] == stock)]
        doc1 = doc1[['field']]
        doc1 = doc1['field'].unique()

        new_doc1 = []
        for i in doc1:
            string = str(i)
            new_doc1.append(string)
        doc1 = new_doc1
        doc1.sort()
        search_options2 = [{'label': i, 'value': i} for i in doc1]
        value = doc1[0]
        
    else:
        # CashFlowStatement
        doc1 = pd.read_csv('Data/Cash_Flow_Statement_Data.csv')
        doc1 = doc1.loc[(doc1['company_name'] == stock)]
        doc1 = doc1[['field']]
        doc1 = doc1['field'].unique()

        new_doc1 = []
        for i in doc1:
            string = str(i)
            new_doc1.append(string)
        doc1 = new_doc1
        doc1.sort()
        search_options2 = [{'label': i, 'value': i} for i in doc1]
        value = doc1[0]

    return search_options2, value

# Header update function
@app.callback([Output('header','children'),
               Output('field_statement','children')],
              [Input('stock','value'),
               Input('rate','value')])

def name(stock, rate):
    
    if rate == 1:
        statement = 'Balance Sheet'
    elif rate == 2:
        statement = 'Income Statement'
    else:
        statement = 'Cash-Flow Statement'

    return "{}'s 20-Year Financials".format(stock), 'Choose a(n) {} Field'.format(statement)

# Live stock data function
@app.callback([Output('price','children'),
               Output('PE','children'),
               Output('EPS','children'),
               Output('EPS%','children'),
               Output('summary','children')],
              [Input('stock','value'),
               Input('rate','value')])

def stock_data(stock, rate):
    
    # Statement type
    if rate == 1:
        # BalanceSheet
        doc1 = pd.read_csv('Data/Balance_Sheet_Data.csv')
        doc1 = doc1.loc[doc1['company_name'] == stock].reset_index(drop = True)
        ticker = doc1.loc[0, 'company_ticker']
        
    elif rate == 2:
        # IncomeStatement
        doc1 = pd.read_csv('Data/Income_Statement_Data.csv')
        doc1 = doc1.loc[doc1['company_name'] == stock].reset_index(drop = True)
        ticker = doc1.loc[0, 'company_ticker']
        
    else:
        # CashFlowStatement
        doc1 = pd.read_csv('Data/Cash_Flow_Statement_Data.csv')
        doc1 = doc1.loc[doc1['company_name'] == stock].reset_index(drop = True)
        ticker = doc1.loc[0, 'company_ticker']

    # Establish web-scraper
    url = 'https://www.marketwatch.com/investing/stock/{}/profile'.format(ticker)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Scrape live stock prices
    price = soup.select('.intraday__data > h3')
    price = [x.get_text().strip() for x in price]
    price = price[0]
    price = re.sub('[^a-zA-Z0-9.-]+', ' ', price)
    price = price.replace(" ", "")

    # Debug
    if price == 'N/A':
        price = '-'
    else:
        price = price

    # Scarpe live pe stock data
    PE = soup.select('.group > .element > table > tbody > tr > td')
    PE = [x.get_text().strip() for x in PE]
    PE = PE[1]

    # Debug
    if PE == 'N/A':
        PE = '-'
    else:
        PE = PE

    # Debug
    try:
        EPS = round(float(price) / float(PE), 2)
    except (ZeroDivisionError, ValueError):
        EPS = '-'

    # Debug
    try:
        EPS_percent = round(float(EPS) / float(price), 2)
    except (ZeroDivisionError, ValueError):
        EPS_percent = '-'

    # Scrape company summary
    summary = soup.select('.column > div > p')
    summary = [x.get_text().strip() for x in summary]
    summary = summary[0]

    # Debug
    if summary == '':
        summary = '-'
    else:
        summary = summary

    EPS = '$' + str(EPS)
    EPS_percent = str(EPS_percent * 100) + '%'

    return '${}'.format(price), PE, EPS, EPS_percent, summary

# Graphics funtion
@app.callback([Output('scat1','figure'),
               Output('scat2','figure'),
               Output('scat3','figure'),
               Output('scat4','figure')],
              [Input('stock','value'),
               Input('rate','value'),
               Input('field','value')])

def graphs(stock, rate, field):
    
    # Statement type
    if rate == 1:
        # BalanceSheet
        doc1 = pd.read_csv('Data/Balance_Sheet_Data.csv')
    elif rate == 2:
         # IncomeStatement
        doc1 = pd.read_csv('Data/Income_Statement_Data.csv')
    else:
        doc1 = pd.read_csv('Data/Cash_Flow_Statement_Data.csv')
    
    doc2 = doc1.loc[(doc1['company_name'] == stock) & (doc1['field'] == field)].reset_index(drop = True)
    X = doc2['year']
    y = round(doc2['value'], 2)
    
    # Create vis
    data1 = go.Figure(data = go.Scatter(x = X,
                                        y = y,
                                        mode = 'lines+markers',
                                        marker = {'size':12,'line':{'width':2,'color':'DarkSlateGrey'},
                                                  'color':'#1E90FF'},
                                        name = 'Shares Outstanding'),

                       layout = go.Layout(paper_bgcolor = 'rgba(0,0,0,0)',
                                          plot_bgcolor = 'rgba(0,0,0,0)',
                                          font = {'color': '#111111'},
                                          height = 260))

    # Set title
    data1.update_layout(title = str(stock) + "'s <br>" + str(field) + ' Over Years',
                        title_font_size = 16, title_font_color = 'Black', title_x = 0.5)

    ###########################################################################################################
    
    # Average DEC closing prices per share
    close = pd.read_csv('Data/Other_Data.csv')
    doc3 = close.loc[(close['company_name'] == stock) & 
                     (close['field'] == 'Price Per Share')].reset_index(drop = True)
    X = doc3['year']
    y = round(doc3['value'], 2)

    # Create vis
    data2 = go.Figure(data = go.Scatter(x = X,
                                        y = y,
                                        mode = 'lines+markers',
                                        marker = {'size':12,'line':{'width':2,'color':'DarkSlateGrey'},
                                                  'color':'#FFA500'},
                                        name = 'Shares Outstanding'),

                      layout = go.Layout(paper_bgcolor = 'rgba(0,0,0,0)',
                                         plot_bgcolor = 'rgba(0,0,0,0)',
                                         font = {'color': '#111111'},
                                         height = 230))

    # Set title
    data2.update_layout(title = str(stock) + "'s <br> Average Price Per Share (DEC) Over Years",
                        title_font_size = 16, title_font_color = 'Black', title_x = 0.5)

    ###########################################################################################################
    
    # Average Annual diluted share outstanding
    so = pd.read_csv('Data/Other_Data.csv')
    doc4 = so.loc[(so['company_name'] == stock) & (so['field'] == 'Shares Outstanding')].reset_index(drop = True)
    X = doc4['year']
    y = round(doc4['value'], 2)

    # Create vis
    data3 = go.Figure(data = go.Scatter(x = X,
                                        y = y,
                                        mode = 'lines+markers',
                                        marker = {'size':12,'line':{'width':2,'color':'DarkSlateGrey'},
                                                  'color':'#FFA500'},
                                        name = 'Shares Outstanding'),

                       layout = go.Layout(paper_bgcolor = 'rgba(0,0,0,0)',
                                          plot_bgcolor = 'rgba(0,0,0,0)',
                                          font = {'color': '#111111'},
                                          height = 230))

    # Set title
    data3.update_layout(title = str(stock) + "'s <br> Shares Outstanding Over Years",
                        title_font_size = 16, title_font_color = 'Black', title_x = 0.5)

    ###########################################################################################################
    
    # Average annual market-cap
    mc = pd.read_csv('Data/Other_Data.csv')
    doc5 = mc.loc[(mc['company_name'] == stock) & (mc['field'] == 'Market Capitalization')].reset_index(drop = True)
    X = doc5['year']
    y = round(doc5['value'], 2)

    # Create vis
    data4 = go.Figure(data = go.Scatter(x = X,
                                        y = y,
                                        mode = 'lines+markers',
                                        marker = {'size':12,'line':{'width':2,'color':'DarkSlateGrey'},
                                                  'color':'#FFA500'},
                                        name = 'Shares Outstanding'),

                       layout = go.Layout(paper_bgcolor = 'rgba(0,0,0,0)',
                                          plot_bgcolor = 'rgba(0,0,0,0)',
                                          font = {'color': '#111111'},
                                          height = 230))
    
    # Set title
    data4.update_layout(title = str(stock) + "'s <br> Market Capitalization Over Years",
                        title_font_size = 16, title_font_color = 'Black', title_x = 0.5)

    return data1, data2, data3, data4

@app.callback(Output('output', 'children'),
              [Input('submit-val', 'n_clicks')],
               state = [State('input1', 'value'),
                        State('input2', 'value'),
                        State('input3', 'value')])

def compute(n_clicks, input1, input2, input3):
    
    if input1 == None:
        input1 = 1
    if input2 == None:
        input2 = 1
    if input3 == None:
        input3 = 1
    
    solution = npf.rate(nper = float(input3), pmt = 0, pv = float(input1) * -1, fv = float(input2))
    solution = round(solution, 2) * 100
    
    return '{}%'.format(solution)

if __name__ == '__main__':
    app.run_server()